### Process the real (raw) Merian data
1. Scale the image to `ZP = 27.0` (same as HSC).
2. Subtract background. **TODO: Have to consider the effect of background subtraction and image scaling on the weight map.**
3. Extract PSF at a given position
    
    Download and install `psfex`: https://github.com/esheldon/psfex

    Documentation: https://psfex.readthedocs.io/en/latest/GettingStarted.html

In [1]:
%load_ext autoreload
%autoreload 2

In [61]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table, Column, hstack, vstack, QTable
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import download_file, clear_download_cache

import kuaizi
from kuaizi.display import display_single, SEG_CMAP
from kuaizi.download import download_decals_cutout
from kuaizi import DECaLS_pixel_scale, DECaLS_zeropoint

import pickle
from IPython.display import clear_output

In [3]:
import sep
from kuaizi.utils import save_to_fits
import os
import sys

In [5]:
kuaizi.set_env(project='Merian', name='', data_dir='/scratch/gpfs/jiaxuanl/Data')

'/scratch/gpfs/jiaxuanl/Data/Merian/'

In [125]:
obs_cat = QTable.read('./Catalogs/magellan/cosmos_imacs_z.fits')
cosmos_cat = QTable.read('./Catalogs/magellan/COSMOS_IMACS_targets.fits')

flag = np.in1d(cosmos_cat['OBJID'].astype('S16').data, obs_cat['id'].data)
obs_cat = obs_cat[np.in1d(obs_cat['id'].data, cosmos_cat[flag]['OBJID'])]
cosmos_cat = cosmos_cat[flag]

obs_cat['id'] = obs_cat['id'].astype(int)

obs_cat.sort('id')
cosmos_cat.sort('OBJID')

assert np.sum(cosmos_cat['OBJID'].data != obs_cat['id']) == 0, 'two catalogs do not match'
obj_cat = hstack([cosmos_cat, obs_cat])
obj_cat['half_size'] = np.ones(len(obj_cat)) * 10 * u.arcsec
obj_cat.rename_column('RA', 'ra')
obj_cat.rename_column('DEC', 'dec')
obj_cat.rename_column('OBJID', 'name')
obj_cat.write('./Cutout/magellan_spec/magellan_spec_obj_cat.fits', overwrite=True)

---

In [137]:
obj_cat = Table.read('./Cutout/magellan_spec/magellan_spec-2022-04-20_n708_done.fits')
obj_cat.remove_column('PREFIX')
obj_cat.write('./Cutout/magellan_spec/magellan_spec-2022-04-20_n708_test.fits', overwrite=True)

### Also need to generate HSC cutouts

In [139]:
sys.path.append('/home/jiaxuanl/Research/Merian/merian_tractor/scripts')
from s18a_batch_cutout import batch_cutout

In [152]:
DATADIR="/scratch/gpfs/jiaxuanl/Data/"
LSBGDIR="/scratch/gpfs/jiaxuanl/Data/Merian/"

In [ ]:
batch_cutout(DATADIR, 
             LSBGDIR + "/Cutout/magellan_spec/magellan_spec-2022-04-20_n708_test.fits", 
             low=0, high=None, size='half_size', unit='arcsec', 
             bands='grizy', name='name', prefix='s18a_wide', njobs=3,
             overwrite=False)


 Number of jobs: 3

 Number of galaxies: 759
    - Generate cutouts for 759 galaxies in g-band.
    - Generate cutouts for 759 galaxies in r-band.
    - Generate cutouts for 759 galaxies in i-band.
    - Generate cutouts for 759 galaxies in z-band.


In [144]:
obj = Table.read('./Catalogs/s18a_wide_cutout_cat_.fits')[0]
obj

name,dir,chunk,ra,dec,half_size,cutout_size,prefix,RA,DEC,radius,image_flag [5],psf_flag [5]
,,,,,arcsec,,,,,arcsec,,
int64,bytes65,int64,float64,float64,float64,float64,bytes25,float64,float64,float64,bool,bool
296216,/scratch/gpfs/jiaxuanl/Data/Merian/Cutout/magellan_spec/1/296216,1,149.63458645739482,1.748167461843215,10.0,10.0,./Cutout/s18a_wide_296216,149.63458645739482,1.748167461843215,10.0,False .. False,False .. False


In [150]:
obj['prefix'].replace('./Cutout/', './Cutout/' +
                        obj['dir'].replace(
                            '/scratch/gpfs/jiaxuanl/Data/Merian/Cutout/', '') + '/')

'./Cutout/magellan_spec/1/296216/s18a_wide_296216'

In [25]:
# N708
for obj in obj_cat[:]:
    file = os.path.join(obj['dir'], f'{obj["PREFIX"]}_N708_deepCoadd_calexp.fits')
    newfile = file.replace('/projects/MERIAN/poststamps/g09_broadcut_new', 
                           '/scratch/gpfs/jiaxuanl/Data/Merian/Cutout/_new/')
    os.system(f'cp {file} {newfile}')
    
    file = os.path.join(obj['dir'], f'{obj["PREFIX"]}_N708_psf.fits')
    newfile = file.replace('/projects/MERIAN/poststamps/g09_broadcut_new', 
                           '/scratch/gpfs/jiaxuanl/Data/Merian/Cutout/_new/')
    os.system(f'cp {file} {newfile}')

In [26]:
# N540
for obj in obj_cat[:]:
    file = os.path.join(obj['dir'], f'{obj["PREFIX"]}_n540_deepCoadd_calexp.fits')
    file = file.replace('g09_broadcut_new', 'g09_broadcut')
    newfile = file.replace('/projects/MERIAN/poststamps/g09_broadcut', 
                           '/scratch/gpfs/jiaxuanl/Data/Merian/Cutout/_new/')
    newfile = newfile.replace('n540', 'N540')
    os.system(f'cp {file} {newfile}')
    
    file = os.path.join(obj['dir'], f'{obj["PREFIX"]}_n540_psf.fits')
    file = file.replace('g09_broadcut_new', 'g09_broadcut')
    newfile = file.replace('/projects/MERIAN/poststamps/g09_broadcut', 
                           '/scratch/gpfs/jiaxuanl/Data/Merian/Cutout/_new/')
    newfile = newfile.replace('n540', 'N540')
    os.system(f'cp {file} {newfile}')

In [49]:
obj

ID,PREFIX,dir,chunk,ra,dec,half_size,name,N708_tractor_flux,g_tractor_flux,r_tractor_flux,i_tractor_flux,z_tractor_flux,y_tractor_flux,tractor_sersic_index,tractor_re,g_gaap_mag,r_gaap_mag,i_gaap_mag,z_gaap_mag,y_gaap_mag,N708_gaap_mag,N540_gaap_mag,z_cosmos,mass_cosmos,ID_yifei
,,,,,,arcsec,,,,,,,,,,,,,,,,,,,
int64,bytes30,bytes65,int64,float64,float64,float64,bytes26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
43158584464269160,g09_broadcut_43158584464269160,/projects/MERIAN/poststamps/g09_broadcut_new/21/43158584464269160,21,150.24015958954186,1.6278470294283613,8.966204135530264,cosmos_43158584464269160_y,0.31933787339460457,0.27624915307852543,0.5774337596535115,0.7590125450373362,0.9010435911305796,1.0549070561963656,0.0,0.46970513203725595,23.851113120039912,23.035300837876484,22.781219061594122,22.529853898129744,22.52245767517013,22.88666185808163,23.353810700829325,0.3319000005722046,9.04985,230148
